In [1]:
# fix front bbox rotation: currently no rotations possible thus whole front radius is used

In [2]:
import numpy as np
from Mapping.BoundingBox import BoundingBox
from Mapping.Cell import Cell
import plotly.graph_objects as go
import json
import time

In [3]:
with open("./config.json", 'r') as f:
    config = json.load(f)

In [4]:
config_front = config['agents'][0]['sensors'][-2]["configuration"]
config_depth = config['agents'][0]['sensors'][-1]["configuration"]
theta_depth = np.linspace(-config_depth["Azimuth"]/2, config_depth["Azimuth"]/2, config_depth["AzimuthBins"])*np.pi/180

In [5]:
front_ak = config_front['RangeMax'] / np.cos(np.deg2rad(config_front["Azimuth"]/2.))
boundary_relativ_front : BoundingBox = BoundingBox((-config_front['RangeMax'], -config_front['RangeMax'], 0), (config_front['RangeMax'], config_front['RangeMax'], 0)) # x,y. z is fixed

z_tmp = np.cos(theta_depth)*config_depth['RangeMax']
z_rel_uncovered = [z_tmp[0], z_tmp[-1]]
z_rel_covered = z_tmp[1:-1]
del z_tmp

y_tmp = np.sin(theta_depth)*config_depth['RangeMax']
y_rel_covered = y_tmp[1:-1]
y_rel_uncovered = [y_tmp[0], y_tmp[-1]]
del y_tmp

x_rel_covered = np.sin(np.deg2rad(config_depth["Elevation"]))*config_depth['RangeMax']
depth_gk = z_rel_uncovered[0]
depth_ak = y_rel_uncovered[-1]
delta = 0.0
boundary_relativ_depth : BoundingBox = BoundingBox((-x_rel_covered - delta, -depth_ak - delta, -config_depth['RangeMax'] - delta), (x_rel_covered + delta, depth_ak + delta, -config_depth['RangeMin'] + delta))
theta_front = np.linspace(-config_front["Azimuth"]/2, config_front["Azimuth"]/2, config_front["AzimuthBins"])*np.pi/180

In [6]:
def _init_debug():
        fig = go.FigureWidget()
        fig.add_mesh3d(
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            opacity=.4,
            color='#DC143C',
            flatshading = True,
            showlegend=True,
            name="bbox"
        )
        
        fig.add_scatter3d(
            mode='markers',
            marker=dict(
                size=2,
                color="black"
            ),
            showlegend=True,
            name="points"
        )
        
        fig.update_layout(
            title="bbox debug",
            xaxis_title="y",
            yaxis_title="x", #use same system as holoocean
            scene = dict(
                xaxis = dict(range=[-20,20]),
                yaxis = dict(range=[-20,20]),
                zaxis = dict(range=[-20,20]),
            )
        )
        
        return fig

In [7]:
def _update_collision_debug(fig, bbox, points):        
        with fig.batch_update():
            x,y,z = zip(*points)
            
            fig.data[0].x = [bbox[0][0], bbox[0][0], bbox[1][0], bbox[1][0], bbox[0][0], bbox[0][0], bbox[1][0], bbox[1][0]]
            fig.data[0].y = [bbox[0][1], bbox[1][1], bbox[1][1], bbox[0][1], bbox[0][1], bbox[1][1], bbox[1][1], bbox[0][1]]
            fig.data[0].z = [bbox[0][2], bbox[0][2], bbox[0][2], bbox[0][2], bbox[1][2], bbox[1][2], bbox[1][2], bbox[1][2]]
            
            fig.data[1].x = x
            fig.data[1].y = y
            fig.data[1].z = z

In [8]:
fig = _init_debug()

In [9]:
bbox = boundary_relativ_depth

In [10]:
rob_rot = [0,0,0]

In [11]:
explored_points = [(0, hyp*np.sin(theta), -hyp*np.cos(theta)) for hyp in np.arange(config_depth["RangeMin"], config_depth['RangeMax'], 0.5) for theta in theta_depth[1:-1]]
explored_points = [(np.cos(np.radians(rob_rot[2]))*pos[1] - np.sin(np.radians(rob_rot[2]))*pos[0], - np.sin(np.radians(rob_rot[2]))*pos[1] + np.cos(np.radians(rob_rot[2]))*pos[0] , pos[2]) for pos in explored_points]

In [12]:
_update_collision_debug(fig, (bbox*rob_rot).get_bounding_box(), explored_points)

In [13]:
# rob_rot = [0,0,0]
# while True:
#     explored_points = [(0, hyp*np.sin(theta), -hyp*np.cos(theta)) for hyp in np.arange(config_depth["RangeMin"], config_depth['RangeMax'], 0.5) for theta in theta_depth[1:-1]]
#     explored_points = [(np.cos(np.radians(rob_rot[2]))*pos[1] - np.sin(np.radians(rob_rot[2]))*pos[0], np.sin(np.radians(rob_rot[2]))*pos[1] + np.cos(np.radians(rob_rot[2]))*pos[0] , pos[2]) for pos in explored_points]
    
#     rob_rot[2] += 5
#     print("\rWinkel: {}".format(rob_rot[2]), end="")
#     _update_collision_debug(fig, (boundary_relativ_depth*rob_rot).get_bounding_box(), explored_points)
#     time.sleep(.2)

In [14]:
fig

FigureWidget({
    'data': [{'color': '#DC143C',
              'flatshading': True,
              'i': [7, 0, …

In [15]:
bbox = boundary_relativ_front

In [16]:
triangle_depth_side = [(0, hyp*np.sin(theta), -hyp*np.cos(theta)) for hyp in np.arange(config_depth["RangeMin"],config_depth['RangeMax'], 0.5) for theta in [theta_depth[0], theta_depth[-1]]]
triangle_depth_bottom = [(0, y, -z_rel_uncovered[0]) for y in np.append(y_rel_covered, y_rel_uncovered)]

unex_depth = triangle_depth_bottom + triangle_depth_side
unex_front = [(hyp*np.cos(theta), -hyp*np.sin(theta), 0) for hyp in np.arange(config_depth["RangeMin"], config_depth['RangeMax'], 0.5) for theta in theta_front[1:-1]]

all_unexplored_points = [(np.cos(np.radians(rob_rot[2]))*pos[1] - np.sin(np.radians(rob_rot[2]))*pos[0], np.sin(np.radians(rob_rot[2]))*pos[1] + np.cos(np.radians(rob_rot[2]))*pos[0] , pos[2]) for pos in unex_depth + unex_front]

br_rot = boundary_relativ_front*rob_rot
bd_rot = boundary_relativ_depth*rob_rot

#unexplored_relative_bbox = BoundingBox.min_bbox(br_rot, bd_rot)
unexplored_relative_bbox = boundary_relativ_front

In [17]:
unexplored_points = all_unexplored_points
_update_collision_debug(fig, ((unexplored_relative_bbox)).get_bounding_box(), unexplored_points)

In [18]:
rob_rot = [0,0,0]
while True:
    triangle_depth_side = [(0, hyp*np.sin(theta), -hyp*np.cos(theta)) for hyp in np.arange(config_depth["RangeMin"],config_depth['RangeMax'], 0.5) for theta in [theta_depth[0], theta_depth[-1]]]
    triangle_depth_bottom = [(0, y, -z_rel_uncovered[0]) for y in np.append(y_rel_covered, y_rel_uncovered)]

    unex_depth = triangle_depth_bottom + triangle_depth_side
    unex_front = [(hyp*np.cos(theta), -hyp*np.sin(theta), 0) for hyp in np.arange(config_depth["RangeMin"], config_depth['RangeMax'], 0.5) for theta in theta_front[1:-1]]

    unexplored_points = [(np.cos(np.radians(rob_rot[2]))*pos[1] - np.sin(np.radians(rob_rot[2]))*pos[0], np.sin(np.radians(rob_rot[2]))*pos[1] + np.cos(np.radians(rob_rot[2]))*pos[0] , pos[2]) for pos in unex_depth + unex_front]

    unexplored_relative_bbox = BoundingBox.min_bbox(boundary_relativ_depth, boundary_relativ_front)
    #unexplored_relative_bbox = boundary_relativ_depth
    a1 = (boundary_relativ_depth*rob_rot)
    a2 = (boundary_relativ_front)
    t =  BoundingBox.min_bbox(a1, a2)

    rob_rot[2] += 1
    print("\rWinkel: {}".format(rob_rot[2]), end="")
    _update_collision_debug(fig, t.get_bounding_box(), unexplored_points)
    time.sleep(0.2)

Winkel: 26

KeyboardInterrupt: 

In [19]:
(boundary_relativ_front*rob_rot).get_bounding_box()

[[-9.208457990201792, -26.743303861764886, 0],
 [9.208457990201792, 26.743303861764886, 0]]

In [20]:
rob_rot

[0, 0, 26]

In [21]:
(boundary_relativ_front*rob_rot).get_bounding_box()

[[-9.208457990201792, -26.743303861764886, 0],
 [9.208457990201792, 26.743303861764886, 0]]

In [22]:
theta_front

array([-1.04719755, -0.93696623, -0.82673491, -0.71650359, -0.60627227,
       -0.49604095, -0.38580962, -0.2755783 , -0.16534698, -0.05511566,
        0.05511566,  0.16534698,  0.2755783 ,  0.38580962,  0.49604095,
        0.60627227,  0.71650359,  0.82673491,  0.93696623,  1.04719755])

In [23]:
((unexplored_relative_bbox)*rob_rot).get_bounding_box()

[[-9.208457990201792, -26.743303861764886, -20.0],
 [9.208457990201792, 26.743303861764886, 0]]

In [24]:
br_rot.get_bounding_box()

[[-20.0, -20.0, 0], [20.0, 20.0, 0]]

In [25]:
bd_rot.get_bounding_box()

[[-17.32050807568877, -0.08726618569493141, -20.0],
 [17.32050807568877, 0.08726618569493141, -0.5]]

In [26]:
unexplored_relative_bbox.get_bounding_box()

[(-20, -20, -20.0), (20, 20, 0)]

In [27]:
a1 = (boundary_relativ_depth*rob_rot)

In [28]:
a2 = (boundary_relativ_front*rob_rot)

In [29]:
BoundingBox.min_bbox(a1,a2).get_bounding_box()

[(-15.529314559406714, -26.743303861764886, -20.0),
 (15.529314559406714, 26.743303861764886, 0)]

In [30]:
(BoundingBox.min_bbox(boundary_relativ_depth, boundary_relativ_front)*rob_rot).get_bounding_box()

[[-9.208457990201792, -26.743303861764886, -20.0],
 [9.208457990201792, 26.743303861764886, 0]]

In [31]:
a = [(1,2),(3,4)]

In [32]:
if a[0][0] < a[1][0]:
    a[0] = list(a[0])
    a[1] = list(a[1])

    a[0][0], a[1][0] = a[1][0], a[0][0]
    tuple(a[0])
    tuple(a[1])

In [33]:
box = a1.get_bounding_box()

In [34]:
def switch(bbox, i):
    bbox[0][0], bbox[1][0] = bbox[1][0], bbox[0][0]
    return bbox

In [35]:
if box[0][0] > box[1][0]:
    box = switch(box, 0)


In [36]:
(a1*[0,0,90]).get_bounding_box()

[[-15.529314559406714, -7.671245316255005, -20.0],
 [15.529314559406714, 7.671245316255005, -0.5]]